In [1]:
import pandas as pd
import os

# Get a list of all Excel files in the current directory that start with 'API_'
excel_files = [f for f in os.listdir('/content/') if f.startswith('API_') and f.endswith('.xls')]

# Initialize an empty list to store DataFrames for each country
all_countries_2020_data = []

for file in excel_files:
    file_path = os.path.join('/content/', file)

    # Extract country code from the filename (e.g., 'MAR' from 'API_MAR_DS2_en_excel_v2_7562.xls')
    country_code = file.split('_')[1]

    try:
        # Read the Excel file. World Bank data often has metadata in the first few rows.
        # The actual data usually starts after a few rows, and is typically in the 'Data' sheet.
        # I'll try to read the first sheet and assume the data starts at row 4 (0-indexed).
        df = pd.read_excel(file_path, sheet_name='Data', skiprows=3)

        # Rename relevant columns for easier access
        # Assuming columns like 'Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'
        # and then years as column headers.

        # Identify year columns (e.g., '1960', '1961', ..., '2020', etc.)
        # World Bank data often has some non-year columns before the year columns start.
        # Let's assume the first 4 columns are identifiers and the rest are years.
        id_vars = ['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code']

        # Make sure the id_vars exist in the dataframe, if not, adjust.
        # Some older files might have different header structures, handle this generically.
        if all(col in df.columns for col in id_vars):
            # Melt the DataFrame to transform year columns into 'Year' and 'Value' columns
            df_melted = df.melt(id_vars=id_vars, var_name='Year', value_name='Value')

            # Filter for the year 2020
            # 'Year' column might be numeric or string, so convert to string for consistent comparison
            df_2020 = df_melted[df_melted['Year'].astype(str) == '2020'].copy()

            # Add a 'Country_Code' column for easier identification later
            df_2020['Country_Code'] = country_code

            all_countries_2020_data.append(df_2020)
        else:
            print(f"Skipping {file} due to unexpected column structure.")

    except Exception as e:
        print(f"Could not process file {file}: {e}")

# Concatenate all DataFrames into a single DataFrame
if all_countries_2020_data:
    df_statistics_2020 = pd.concat(all_countries_2020_data, ignore_index=True)

    # Display the first few rows of the final DataFrame
    print("\nCombined Statistics for 2020:")
    display(df_statistics_2020.head())

    # Display some general info about the combined dataframe
    print("\nDataFrame Info:")
    df_statistics_2020.info()

    print("\nUnique Countries in 2020 Data:")
    print(df_statistics_2020['Country_Code'].unique())

else:
    print("No data was processed for 2020.")



Combined Statistics for 2020:


,Country Name,Country Code,Indicator Name,Indicator Code,Year,Value,Country_Code
0,Morocco,MAR,Domestic private health expenditure per capita...,SH.XPD.PVTD.PC.CD,2020,99.705928,MAR
1,Morocco,MAR,Out-of-pocket expenditure (% of current health...,SH.XPD.OOPC.CH.ZS,2020,41.287483,MAR
2,Morocco,MAR,Domestic general government health expenditure...,SH.XPD.GHED.GD.ZS,2020,2.465519,MAR
3,Morocco,MAR,External health expenditure (% of current heal...,SH.XPD.EHEX.CH.ZS,2020,2.136761,MAR
4,Morocco,MAR,Newborns protected against tetanus (%),SH.VAC.TTNS.ZS,2020,90.000000,MAR



DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31794 entries, 0 to 31793
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Country Name    31794 non-null  object 
 1   Country Code    31794 non-null  object 
 2   Indicator Name  31794 non-null  object 
 3   Indicator Code  31794 non-null  object 
 4   Year            31794 non-null  object 
 5   Value           18046 non-null  float64
 6   Country_Code    31794 non-null  object 
dtypes: float64(1), object(6)
memory usage: 1.7+ MB

Unique Countries in 2020 Data:
['MAR' 'BHR' 'SAU' 'QAT' 'IRQ' 'DZA' 'SYR' 'YEM' 'LBN' 'SOM' 'EGY' 'PSE'
 'TUN' 'JOR' 'MRT' 'SDN' 'KWT' 'LBY' 'OMN' 'ARE' 'DJI']


In [5]:
# Create a pivot table with Indicator Name as columns
pivot_table_2020 = df_statistics_2020.pivot_table(index='Country Name', columns='Indicator Name', values='Value')

# Display the first few rows and columns of the pivot table
print("\nPivot Table for 2020 Statistics with Indicator Name as columns:")
display(pivot_table_2020.head())


Pivot Table for 2020 Statistics with Indicator Name as columns:


Indicator Name,ARI treatment (% of children under 5 taken to a health provider),Access to clean fuels and technologies for cooking (% of population),"Access to clean fuels and technologies for cooking, rural (% of rural population)","Access to clean fuels and technologies for cooking, urban (% of urban population)",Access to electricity (% of population),"Access to electricity, rural (% of rural population)","Access to electricity, urban (% of urban population)",Adjusted net national income (annual % growth),Adjusted net national income (constant 2015 US$),Adjusted net national income (current US$),...,"Wage and salaried workers, female (% of female employment) (modeled ILO estimate)","Wage and salaried workers, male (% of male employment) (modeled ILO estimate)","Wage and salaried workers, total (% of total employment) (modeled ILO estimate)","Water productivity, total (constant 2015 US$ GDP per cubic meter of total freshwater withdrawal)",Wholesale price index (2010 = 100),Women Business and the Law Index Score (scale 1-100),Women who were first married by age 15 (% of women ages 20-24),Women who were first married by age 18 (% of women ages 20-24),Women's share of population ages 15+ living with HIV (%),Young people (ages 15-24) newly infected with HIV
Country Name,,,,,,,,,,,,,,,,,,,,,
Algeria,NaN,99.7,98.9,99.9,99.7,99.1,99.9,-6.141459,1.325856e+11,1.184676e+11,...,74.773837,66.606404,67.827568,18.081271,NaN,57.500,NaN,NaN,47.127746,550.0
Bahrain,NaN,100.0,100.0,100.0,100.0,100.0,100.0,-8.577098,2.369217e+10,2.539957e+10,...,96.639881,97.503140,97.329685,223.886358,NaN,55.625,NaN,NaN,13.838432,100.0
Djibouti,NaN,10.1,0.2,12.8,64.5,35.8,72.6,3.291667,2.694080e+09,2.742440e+09,...,37.289778,52.296024,48.506123,161.322932,NaN,71.250,NaN,NaN,51.037641,100.0
"Egypt, Arab Rep.",NaN,99.9,99.9,99.9,100.0,100.0,100.0,5.092479,3.385004e+11,3.241087e+11,...,70.004052,73.134979,72.644499,5.318878,NaN,45.000,NaN,NaN,16.442011,890.0
Iraq,NaN,99.3,98.9,99.6,100.0,100.0,100.0,-19.368749,1.340520e+11,1.491322e+11,...,63.539188,69.659383,68.946053,3.940482,NaN,45.000,NaN,NaN,19.372942,100.0


In [9]:
pivot_table_2020.to_excel("world_bank_data.xlsx")